In [4]:
import numpy as np
from numba import jit
from numba import njit, prange
import matplotlib.pyplot as plt
import pickle
import pandas as pd
import time
import random
import os
import multiprocessing

# The Ising Model: An NP-Complete Ferromagnetic Model
### This project will focus on the implementation and optimization of the Ising model, a model intended to represent the interactions of ferromagnetic atoms in a condensed material.

In [8]:
def pickler(filename, data_to_save):
    """
    Writes Python data structure in memory to file on disk
    """
    with open(filename, 'wb') as file:
        pickle.dump(data_to_save, file)

def unpickler(filename):
    """
    Reads data on disk to Python data structure in memory
    """
    with open(filename, 'rb') as file:
        output = pickle.load(file)
    return output

def ising_generator(n, random_state=0):
    """
    Generates an nxn array of randomly assigned spin up (1) or spin down (0) elements to represent a 2D ferromagnetic material
        Inputs: 
            n, integer
        Outputs: 
            ising_arr, nxn np.array()
        Errors:
            AssertError: raised if input is not of type int.
    """
    assert type(n) is int, "Input must be of type int."
    ising_arr = np.random.randint(2,size = (n,n))
    ising_arr[ising_arr == 0] = -1
    return ising_arr

@jit(nopython = True)
def config_energy(ising_arr, J):    
    """
    Inputs:
        ising_arr: np.ndarray, randomly distributed 0s and 1s in square matrix
        J: Interaction term, determines if anti-ferromagnetic (+1) or ferromagnetic (-1) or non-interacting (0)
    Outputs:
        final_energy: total energy associated with configuration of ising_arr
    Errors:
        AssertError: ising_arr must be a square array
    """
    assert J == 1 or J == -1 or J == 0 #Validate input for J
    assert ising_arr.shape[0] == ising_arr.shape[1] #Assert square input array
    n = ising_arr.shape[0]
    num_interactions = 0
    hamiltonian = 0 #init hamiltonian summation
    for i in range(n):
        for j in range(n):
            ## Only need to calculate interaction with element below and element to the right to prevent double counting ferromagnetic interactions
            hamiltonian += -J * ising_arr[i][j] * ising_arr[ (i + 1) % n ][j] #Lower interaction, periodic boundary condition is maintained by the remainder calculation in the index
            hamiltonian += -J * ising_arr[i][j] * ising_arr[i][ (j +1 ) % n ] #Right interaction, periodic boundary condition is maintained by the remainder calculation in the index
    hamiltonian = hamiltonian / n**2 #Normalization of total hamiltonian energy to energy/site
    magnetism = np.sum(ising_arr) / n**2 #Total magnetism normalized to magnetization/site
    return hamiltonian, magnetism

@jit(nopython = True)
def single_flip(ising_arr, J, i, j, init_energy):
    """
    Computationally efficient method of calculating change in energy by flipping a site's magnetization 
    in MCMC and only looking at the change in the local environment's Hamiltonian energy, assumption of negligible 
    long-range interactions. 
    Inputs:
        ising_arr: square np.array() from ising_generator()
        J: Interaction term, int of either -1, 0, 1
        i: int, row index to flip
        j: int, column index to flip
        init_energy: float-like, energy of configuration before flip
    Outputs: 
        delta_e: Potential change in configurational energy as a result of the element's flip in magnetism
        new_e: Configurational energy following the flip in magnetism
    """
    n = ising_arr.shape[0]
    flipped_site = -ising_arr[i][j] #Reverse charge on site 
    delta_e = 0
    delta_e += -J * flipped_site * ising_arr[(i+1) % n][j] #upper interaction
    delta_e += -J * flipped_site * ising_arr[(i-1) % n][j] #lower interaction
    delta_e += -J * flipped_site * ising_arr[i][(j + 1) % n] #right interaction
    delta_e += -J * flipped_site * ising_arr[i][(j - 1) % n] # left interaction

    new_e = init_energy + delta_e

    return delta_e, new_e

#@jit(nopython = True)
def markovchain_montecarlo(ising_arr, beta, mcmc_iterations, equilibriating_iterations, J=-1): #old function that doesn't work for some reason
    """
    Wrapper for a single Markov-Chain Monte Carlo optimization of a 2D Ising lattice.
    Inputs:
        ising_arr: Array to optimize from ising_generator function
        beta: Temperature constant, float
        mcmc_iterations: int, number of MCMC iterations to run following equilibriation
        equilibriating_iterations: int, number of equilbriating iterations to run before recording the MCMC configurational energies to memory
        savename: str, filename for which to pickle saved data to
        J: int of either -1, 0 or 1, Ising interaction constant  
    Outputs:
        energies: array of configurational energies/step
        magnetism: array of total magnetization/step
    """
    assert ising_arr.shape[0] == ising_arr.shape[1] #must be square array
    n = ising_arr.shape[0] 
    n_square = n**2 #used for normalization
    init_energy = config_energy(ising_arr, J)
    #Equilibriation
    for i in range(equilibriating_iterations):
        #### It is standard practice to run a set of equilbriation steps in order to account for bad initial guesses for a structure
        #### Call MCMC optimization for equilbriating_iterations steps
        #### Do not record energies/magnetism to memory
        rand_i = np.random.randint(0, n)
        rand_j = np.random.randint(0, n)
        output = single_flip(ising_arr, J, rand_i, rand_j, init_energy) #calculate change in energy for altered configuration
        delta_E = output[0]
        new_E = output[1]
        
        chance = np.random.rand()
        # MCMC stochastic condition for accepting new condition
        if delta_E < 0:
            #If negative change in free energy, accept change 
            ising_arr[rand_i, rand_j] *= -1
            init_energy = new_E
        elif chance <= np.exp(-delta_E * beta): 
            #If positive change in free energy and odds are in favor of switch, accept change
            ising_arr[rand_i, rand_j] *= -1
            init_energy = new_E
        elif chance > np.exp(-delta_E * beta): 
            # Positive change in free energy, odds not in favor of switch, reject change
            pass
    
    energy_container = np.zeros(mcmc_iterations) #Preallocate container for MCMC energies
    magnetism_container = np.zeros(mcmc_iterations) #Preallocate container for magnetizations per MCMC step
    for i in range(mcmc_iterations):
        #### Call MCMC optimization for mcmc_iterations steps
        #### Write energies/magnetism to memory for plotting etc.
        energy_container[i] = init_energy
        magnetism_container[i] = np.sum(ising_arr) / n_square #update magnetization

        #Select random site to flip
        rand_i = np.random.randint(0, n)
        rand_j = np.random.randint(0, n)
        output = single_flip(ising_arr, J, rand_i, rand_j, init_energy) #calculate change in energy for altered configuration
        delta_E = output[0]
        new_E = output[1]
        
        chance = np.random.rand() 
        # MCMC stochastic condition for accepting new condition
        if delta_E < 0:
            #If negative change in free energy, accept change 
            ising_arr[rand_i, rand_j] *= -1
            init_energy = new_E
        elif chance <= np.exp(-delta_E * beta): 
            #If positive change in free energy and odds are in favor of switch, accept change
            ising_arr[rand_i, rand_j] *= -1
            init_energy = new_E
        elif chance > np.exp(-delta_E * beta): 
            # Positive change in free energy, odds not in favor of switch, reject change and ising_arr remains unchanged
            pass
n = 10
ising_arr = ising_generator(n)
begin = time.time()
init_energy = config_energy(ising_arr, -1)[0]
beta = 4
a,b = single_flip(ising_arr, -1, 2, 4, init_energy)
mcmc_iterations = 200000
n_square = n**2
J = -1

def mcmc_optimization(equilibriating_iterations, mcmc_iterations, n, beta, J): 
    ## Eventually save the array every 1000 iterations for visualization
    
    energy_container = np.zeros(mcmc_iterations) #Preallocate container for MCMC energies
    magnetism_container = np.zeros(mcmc_iterations) #Preallocate container for magnetizations per MCMC step
    ising_arr = ising_generator(n)
    init_energy = config_energy(ising_arr, J)[0]
    n_square = n**2
    for i in range(equilibriating_iterations):
        #### Call MCMC optimization for equilibriating_iterations steps
        #### Energies/magnetizations not saved to memory during equilibriation
        for i in range(n_square): 
            # One equilibriating iteration considered to be accomplished after flipping n^2 sites. 
            # While this unfortunately creates O(N^2) time complexity, it is necessary to compare lattices of different sizes.
            #Select random site to flip
            rand_i = np.random.randint(0, n)
            rand_j = np.random.randint(0, n)
            output = single_flip(ising_arr, J, rand_i, rand_j, init_energy) #calculate change in energy for altered configuration
            delta_E = output[0]
            new_E = output[1]
            
            chance = np.random.rand() 
            # MCMC stochastic condition for accepting new condition
            if delta_E < 0:
                #If negative change in free energy, accept change 
                ising_arr[rand_i, rand_j] *= -1
                init_energy = new_E
            elif chance <= np.exp(-delta_E * beta): 
                #If positive change in free energy and odds are in favor of switch, accept change
                ising_arr[rand_i, rand_j] *= -1
                init_energy = new_E
            elif chance > np.exp(-delta_E * beta): 
                # Positive change in free energy, odds not in favor of switch, reject change and ising_arr remains unchanged
                pass

    animation_container = []
    animation_save_index = 0
    for i in range(mcmc_iterations):
        #### Call MCMC optimization for mcmc_iterations steps
        #### Write energies/magnetism to memory for plotting etc.
        energy_container[i] = init_energy 
        magnetism_container[i] = np.sum(ising_arr)  #update magnetization
        for i in range(n_square):
            #Select random site to flip
            rand_i = np.random.randint(0, n)
            rand_j = np.random.randint(0, n)
            output = single_flip(ising_arr, J, rand_i, rand_j, init_energy) #calculate change in energy for altered configuration
            delta_E = output[0]
            new_E = output[1]
            
            chance = np.random.rand() 
            # MCMC stochastic condition for accepting new condition
            if delta_E < 0:
                #If negative change in free energy, accept change 
                ising_arr[rand_i, rand_j] *= -1
                init_energy = new_E
            elif chance <= np.exp(-delta_E * beta): 
                #If positive change in free energy and odds are in favor of switch, accept change
                ising_arr[rand_i, rand_j] *= -1
                init_energy = new_E
            elif chance > np.exp(-delta_E * beta): 
                # Positive change in free energy, odds not in favor of switch, reject change and ising_arr remains unchanged
                pass
    
    #Normalization of energies & magnetization to a single site
    energy_container = energy_container / n_square
    magnetism_container = magnetism_container / n_square
    animation_save_index += 1
    if animation_save_index % 1000 == 0:
        animation_container.append(ising_arr)
    return energy_container, magnetism_container, animation_container
#mcmc_optimization(200, mcmc_iterations, n, beta, J)
#x = np.arange(mcmc_iterations)
#plt.plot(x, energy_container)
end = time.time()
print(f"Time to run: {end-begin} seconds") 

Time to run: 0.2904839515686035 seconds


In [11]:
class MCMC_optimization: #
    def __init__(self, n, beta, equil_iter, mcmc_iter, J):
        self.n = n
        self.beta = beta
        self.equil_iter = equil_iter
        self.mcmc_iter = mcmc_iter
        self.J = J
        self.savename = f"{n}_{beta}_{equil_iter}_{mcmc_iter}_{J}"
    
# Parameter grid testing for MCMC configs
j_space = [-1, 1]
n_space = [10, 50, 100, 1000]
beta_space = [.01, .1, .5, 1, 3, 4, 5, 10]
equil_iter_space = [500] # We really shouldn't need more than this
mcmc_iter_space = [1000, 2000, 5000, 10000]

#Defining all 256 configurations to be tested
parameter_space = [] #container holding parameters for all relaxations to be ran
for n in n_space:
    for beta in beta_space:
        for eq in equil_iter_space:
            for mcmc in mcmc_iter_space:
                for J_config in j_space:
                    parameter_space.append(MCMC_optimization(n, beta, eq, mcmc, J_config))
num_runs = len(parameter_space)
for i in range(len(parameter_space)):
    remaining_runs = len(parameter_space) - (i + 1)
    testing_params = parameter_space[i]
    
    n = testing_params.n
    beta = testing_params.beta
    equil_iter = testing_params.equil_iter
    mcmc_iter = testing_params.mcmc_iter
    J = testing_params.J
    savename = testing_params.savename
    begin_time = time.time()
    
    energies, magnetism, animation_arr = mcmc_optimization(equil_iter, mcmc_iter, n, beta, J)
    end_time = time.time()
    runtime = end_time - begin_time
    print(f"Runtime for configuration {savename} was {runtime} seconds, {remaining_runs} relaxations remaining of {num_runs} runs.")
    
    #Dictionary to hold data and information about a run
    run_dict = {}
    run_dict["Runtime"] = runtime
    run_dict["Energies"] = energies
    run_dict["Magnetism"] = magnetism
    run_dict["Animation Arrays"] = animation_arr
    run_dict["params"] = savename
    
    #Save run to file
    pickler(savename, run_dict)
    os.system(f"mv {savename} ./mcmc_runs/")

Runtime for configuration 10_0.01_500_1000_-1 was 0.4406437873840332 seconds, 255 relaxations remaining of 256 runs.
Runtime for configuration 10_0.01_500_1000_1 was 0.4331939220428467 seconds, 254 relaxations remaining of 256 runs.
Runtime for configuration 10_0.01_500_2000_-1 was 0.7258901596069336 seconds, 253 relaxations remaining of 256 runs.
Runtime for configuration 10_0.01_500_2000_1 was 0.7255139350891113 seconds, 252 relaxations remaining of 256 runs.
Runtime for configuration 10_0.01_500_5000_-1 was 1.5922386646270752 seconds, 251 relaxations remaining of 256 runs.
Runtime for configuration 10_0.01_500_5000_1 was 1.580409049987793 seconds, 250 relaxations remaining of 256 runs.
Runtime for configuration 10_0.01_500_10000_-1 was 3.0168418884277344 seconds, 249 relaxations remaining of 256 runs.
Runtime for configuration 10_0.01_500_10000_1 was 3.0838398933410645 seconds, 248 relaxations remaining of 256 runs.
Runtime for configuration 10_0.1_500_1000_-1 was 0.4350900650024414

In [5]:
from multiprocessing import Process


def print_func(continent='Asia'):
    print('The name of continent is : ', continent)

if __name__ == "__main__":  # confirms that the code is under main function
    names = ['America', 'Europe', 'Africa']
    procs = []
    proc = Process(target=print_func)  # instantiating without any argument
    procs.append(proc)
    proc.start()

    # instantiating process with arguments
    for name in names:
        # print(name)
        proc = Process(target=print_func, args=(name,))
        procs.append(proc)
        proc.start()

    # complete the processes
    for proc in procs:
        proc.join()

Traceback (most recent call last):
  File "<string>", line 1, in <module>
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/usr/local/Caskroom/miniforge/base/envs/cs501/lib/python3.9/multiprocessing/spawn.py", line 116, in spawn_main
  File "/usr/local/Caskroom/miniforge/base/envs/cs501/lib/python3.9/multiprocessing/spawn.py", line 116, in spawn_main
Traceback (most recent call last):
  File "<string>", line 1, in <module>
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/usr/local/Caskroom/miniforge/base/envs/cs501/lib/python3.9/multiprocessing/spawn.py", line 116, in spawn_main
  File "/usr/local/Caskroom/miniforge/base/envs/cs501/lib/python3.9/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/usr/local/Caskroom/miniforge/base/envs/cs501/lib/python3.9/multiprocessing/spawn.py", line 126, in _main
    exitcode = _main(fd, parent_sentinel)
  File "/usr/local/Caskroom/mi